<a href="https://colab.research.google.com/github/MThwa/crypto-analytics/blob/main/wallet_token_balances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
from datetime import datetime
from google.cloud import bigquery
import random

etherscan_api_key = "replace_here"
table_id =  "bigtimestudios.currency_analytics.ol_wallet_balance"          ## Replace in production: "bigtimestudios.currency_analytics.ol_wallet_balance"

# documentation here https://docs.google.com/document/d/1URb2pDo2WMru7T1lb1vS5eN6uvQQJDSEZCQHL4ZwjFE/edit?tab=t.0
# New Wallets added 6/6/2025
wallets = [
    "0x772f5eEed371F846FEC87062D08Df8734c7f998A",
    "0x46950Ba8946D7be4594399bcf203fB53E1fd7D37",
    "0x549AD7AF45B89F6688dbC814B0eE7C2BeF5AC1F5",
    "0xC3322716475fbA83bfC057112247a43F1A1F2c4C",
    "0xfc5AAbA5315253cAC8b95815C7b99eB38620d8c1",
    "0x5c81EE1DefF371baBa014111dd684678284af493",
    "0x0297Ed45A5c1D8651f31B1Ced2CBe3c8A0a337d5",
    '0x6CfFf6131E1903e02447e71EF10Efc07c8456E3D',                               # updated 6/16: $OL Marketing and Community
    '0xF3a51772Cb262d901F270a1631C825CD9C5A2860',                               # updated 6/16: $OL Player Reward
    '0xDFeE9636d9B34C59067A5d712617064CB5533E69',                               # updated 6/16: $OL Ecosystem & Treausury
    '0x0A1b154c997b3D5D53155d10c2c017b9211f5030',                               # updated 6/16: $BIGTIME Player Reward
    "0x3691f92F4CA60E85cB6425cC543b0594beb29967",                               # updated 6/16: $BIGTIME Marketing and Community
    '0x5125aC7827239dccE21E18f2187F7Ef091008B74',                               # updated 6/16: $BIGTIME Ecosystem & Treasury
]

bigtime_contract = "0x64Bc2cA1Be492bE7185FAA2c8835d9b824c8a194"
ol_contract = "0x1F57da732A77636D913C9a75d685B26CC85DCC3A"
bftoken_contract = "0x9B784F7Fd6C888463666eb2c05AA6E61628E06B2"                 # updated for bossfighter token

client = bigquery.Client()
today = datetime.utcnow().strftime("%Y-%m-%d")

# retry handler with exponential backoff and jitter
def exponential_backoff_retry(fn, *args, max_retries=5, base_delay=1, max_delay=30, **kwargs):
    for attempt in range(max_retries):
        try:
            return fn(*args, **kwargs)
        except Exception as e:
            wait = min(base_delay * (2 ** attempt), max_delay)
            jitter = random.uniform(0.5, 1.5)
            sleep_time = wait * jitter
            print(f"Retry {attempt+1}/{max_retries} after error: {e} (waiting {sleep_time:.2f}s)")
            time.sleep(sleep_time)
    print(f"All {max_retries} retries failed for {fn.__name__}")
    return 0.0 if 'balance' in fn.__name__ else "?"

# fetch wrapper
def fetch_url(url):
    response = requests.get(url, timeout=10)
    return response.json()

def get_token_balance(wallet_address, contract_address):
    url = f"https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress={contract_address}&address={wallet_address}&tag=latest&apikey={etherscan_api_key}"
    response = exponential_backoff_retry(fetch_url, url)
    if "Max calls per sec rate limit reached" in response.get("result", ""):
        time.sleep(1)
        return get_token_balance(wallet_address, contract_address)
    if response.get("status") == "1" or "result" in response:
        try:
            return int(response["result"]) / 10**18
        except ValueError:
            return 0.0
    return 0.0

def get_eth_balance(wallet_address):
    url = f"https://api.etherscan.io/api?module=account&action=balance&address={wallet_address}&tag=latest&apikey={etherscan_api_key}"
    response = exponential_backoff_retry(fetch_url, url)
    if response.get("status") == "1" or "result" in response:
        try:
            return int(response["result"]) / 10**18
        except ValueError:
            return 0.0
    return 0.0

def get_latest_block():
    url = f"https://api.etherscan.io/v2/api?chainid=1&module=proxy&action=eth_blockNumber&apikey={etherscan_api_key}"
    response = exponential_backoff_retry(fetch_url, url)
    if "result" in response:
        return int(response["result"], 16)
    return "?"

block_number = get_latest_block()
print(f"Table {table_id} exists.")
print(f"Fetching balances for {today} (Block {block_number})...\n")

rows_to_insert = []

for wallet in wallets:
    eth_balance = get_eth_balance(wallet)
    time.sleep(1)
    bigtime_balance = get_token_balance(wallet, bigtime_contract)
    time.sleep(1)
    ol_balance = get_token_balance(wallet, ol_contract)
    time.sleep(1)
    bftoken_balance = get_token_balance(wallet, bftoken_contract)               # updated for bftoken
    time.sleep(1)                                                               # updated for bftoken

    print(f"{today} | Wallet: {wallet} | ETH: {eth_balance:.6f} | BIGTIME: {bigtime_balance:,.6f} | OL: {ol_balance:,.6f} | BFTOKEN: {bftoken_balance:,.6f}")

    rows_to_insert.append({
        "date": today,
        "wallet": wallet,
        "eth_balance": eth_balance,
        "bigtime_balance": bigtime_balance,
        "ol_balance": ol_balance,
        "bftoken_balance": bftoken_balance,                                     # updated for bftoken
    })

if rows_to_insert:
    try:
        exponential_backoff_retry(client.insert_rows_json, table_id, rows_to_insert)
        print("Data successfully inserted into BigQuery!")
    except Exception as e:
        print("BigQuery Insert Error:", e)
else:
    print("⚠️ No data to insert.")
